##**07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)**


###**문서 군집화 개념**
**문서 군집화(Clustering)** : 비슷한 텍스트 구성의 문서를 군집화하는 것
- 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로 텍스트 분류와 유사
- 텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요하나, 문서 군집화는 학습 데이터 세트가 필요 없는 비지도학습 기반으로 동작

###**Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기**
- UCI 머신러닝 리포지토리 - 오픈 리뷰 데이터셋
  - 51개의 텍스트 파일
    - Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서

In [6]:
import pandas as pd
import glob, os

path = r'/content/drive/MyDrive/ESAA/23-1/topics'
all_files = glob.glob(os.path.join(path,"*.data"))
filename_list=[]
opinion_text=[]

#개별 파일의 파일명은 filename_list로 취합
#개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  #개별 파일을 읽어 DataFrame으로 생성
  df = pd.read_table(file_, index_col = None, header=0, encoding='latin1')

  #절대 경로로 주어진 파일명을 가공, 리눅스에서 수행할 때는 다음 \\를 /로 변경
  #맨 마지막 .data 확장자도 제거
  filename_ = file_.split('/')[-1]
  filename = filename_.split('.')[0]

  #파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

#파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename': filename_list, 'opinion_text' : opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,keyboard_netbook_1005ha,...
2,voice_garmin_nuvi_255W_gps,...
3,video_ipod_nano_8gb,...
4,display_garmin_nuvi_255W_gps,...


- 파일 이름으로 의견(opinion)의 텍스트(text)가 어떠한 제품/서비스에 대한 리뷰인지 잘 알 수 있음
- TF-IDF 형태로 피처 벡터화
  - tokenizer : LemNormalize() 함수 이용
  - ngram : (1,2)
  - min_df, max_df 범위 설정

In [10]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# 특수 문자 제거 및 단어 원형 추출
def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)

    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)

    # 단어 원형 추출
    return LemTokens(word_tokens)

In [11]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85)

#opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


###**K-means clustering**


In [18]:
from sklearn.cluster import KMeans

#5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters = 5, max_iter = 10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [19]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,4
1,keyboard_netbook_1005ha,...,0
2,voice_garmin_nuvi_255W_gps,...,1
3,video_ipod_nano_8gb,...,1
4,display_garmin_nuvi_255W_gps,...,1


- 그룹 0 : 킨들, 넷북 등 포터블 전자기기 리뷰

In [16]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,buttons_amazon_kindle,...,0
1,keyboard_netbook_1005ha,...,0
8,size_asus_netbook_1005ha,...,0
11,speed_windows7,...,0


- 그룹 1 : 차량용 네비게이션

In [20]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
10,accuracy_garmin_nuvi_255W_gps,...,1
5,directions_garmin_nuvi_255W_gps,...,1
4,display_garmin_nuvi_255W_gps,...,1
27,eyesight-issues_amazon_kindle,...,1
14,features_windows7,...,1
18,fonts_amazon_kindle,...,1
31,navigation_amazon_kindle,...,1
35,price_amazon_kindle,...,1
16,satellite_garmin_nuvi_255W_gps,...,1
12,screen_garmin_nuvi_255W_gps,...,1


- 그룹 2 : 토요타, 혼다 등 자동차 리뷰

In [21]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
47,comfort_honda_accord_2008,...,2
24,comfort_toyota_camry_2007,...,2
23,gas_mileage_toyota_camry_2007,...,2
37,interior_honda_accord_2008,...,2
22,interior_toyota_camry_2007,...,2
44,mileage_honda_accord_2008,...,2
40,performance_honda_accord_2008,...,2
49,quality_toyota_camry_2007,...,2
43,seats_honda_accord_2008,...,2
45,transmission_toyota_camry_2007,...,2


- 그룹 3 : 호텔 리뷰

In [22]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
41,bathroom_bestwestern_hotel_sfo,...,3
32,food_holiday_inn_london,...,3
36,food_swissotel_chicago,...,3
50,free_bestwestern_hotel_sfo,...,3
38,location_bestwestern_hotel_sfo,...,3
19,location_holiday_inn_london,...,3
42,parking_bestwestern_hotel_sfo,...,3
20,price_holiday_inn_london,...,3
25,room_holiday_inn_london,...,3
39,rooms_bestwestern_hotel_sfo,...,3


- 그룹 4 : 포터블 전자기기

In [23]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
34,battery-life_amazon_kindle,...,4
0,battery-life_ipod_nano_8gb,...,4
7,battery-life_netbook_1005ha,...,4
21,performance_netbook_1005ha,...,4


- 군집 개수가 많게 설정되어 있어, 세분화되어 군집화 된 경향이 있음
- 중심 개수를 3개로 낮춰서 결과 확인
---


In [24]:
from sklearn.cluster import KMeans

#3개의 집합으로 군집화
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

#소속 군집을 cluster_label 칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
40,performance_honda_accord_2008,...,0
47,comfort_honda_accord_2008,...,0
21,performance_netbook_1005ha,...,0
43,seats_honda_accord_2008,...,0
22,interior_toyota_camry_2007,...,0
44,mileage_honda_accord_2008,...,0
45,transmission_toyota_camry_2007,...,0
23,gas_mileage_toyota_camry_2007,...,0
37,interior_honda_accord_2008,...,0
49,quality_toyota_camry_2007,...,0


- 자동차 리뷰

In [25]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
47,comfort_honda_accord_2008,...,0
24,comfort_toyota_camry_2007,...,0
23,gas_mileage_toyota_camry_2007,...,0
37,interior_honda_accord_2008,...,0
22,interior_toyota_camry_2007,...,0
44,mileage_honda_accord_2008,...,0
40,performance_honda_accord_2008,...,0
21,performance_netbook_1005ha,...,0
49,quality_toyota_camry_2007,...,0
43,seats_honda_accord_2008,...,0


- 포터블 전자기기 리뷰

In [26]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
10,accuracy_garmin_nuvi_255W_gps,...,1
34,battery-life_amazon_kindle,...,1
0,battery-life_ipod_nano_8gb,...,1
7,battery-life_netbook_1005ha,...,1
26,buttons_amazon_kindle,...,1
5,directions_garmin_nuvi_255W_gps,...,1
4,display_garmin_nuvi_255W_gps,...,1
27,eyesight-issues_amazon_kindle,...,1
14,features_windows7,...,1
18,fonts_amazon_kindle,...,1


- 호텔 리뷰

In [27]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
41,bathroom_bestwestern_hotel_sfo,...,2
32,food_holiday_inn_london,...,2
36,food_swissotel_chicago,...,2
50,free_bestwestern_hotel_sfo,...,2
38,location_bestwestern_hotel_sfo,...,2
19,location_holiday_inn_london,...,2
42,parking_bestwestern_hotel_sfo,...,2
20,price_holiday_inn_london,...,2
25,room_holiday_inn_london,...,2
39,rooms_bestwestern_hotel_sfo,...,2


###**군집별 핵심 단어 추출하기**
`clusters_centers_` : 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치하고 있는지 제공
- 배열 값으로 제공됨
- 행은 개별 군집을, 열은 개별 피처를 의미
  - 배열 내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값

In [28]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.00529646 0.00084138 0.         ... 0.         0.         0.        ]
 [0.00804456 0.         0.         ... 0.00735716 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


- (3,4611) 배열, 군집 3개, 피처 2409개로 구성
- 각 행의 배열 값은 각 군집 내의 4611개의 피처 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것
- 0~1까지의 값을 가질 수 있으며 1에 가까울수록 중심과 가까운 값을 의미

---
cluster_centers_ 속성은 넘파이의 ndarray
- `argsort()[:,::-1]` : 배열 내 값이 큰 순으로 정렬된 위치 인덱스 값 반환

- `get_cluster_details()` : cluster_centers_ 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤, 해당 인덱스를 이용해 핵심 단어 이름과 그 때의 상대 위치 값을 추출해 cluster_details라는 객체 변수에 기록하고 반환하는 것

In [29]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함.
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화.
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함.
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]

        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [30]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

In [31]:
feature_names=tfidf_vect.get_feature_names_out()

cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'performance', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'quality']
Reviews 파일명 : ['performance_netbook_1005ha', 'interior_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'comfort_toyota_camry_2007', 'interior_honda_accord_2008', 'performance_honda_accord_2008', 'seats_honda_accord_2008']
####### Cluster 1
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'kindle', 'direction', 'life', 'video', 'voice', 'feature']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'keyboard_netbook_1005ha', 'voice_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'display_garmin_nuvi_255W_gps', 'directions_garmin_nuvi_255W_gps', 'updates_garmin_nuvi_255W_gps']
####### Cluster 2
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['location_holiday_inn_london', 'price_holiday_inn_london', 'room_holiday_inn_london', 'rooms_swissotel_chicago'

- `cluster 0` : 실내 인테리어, 좌석, 연료 효율 등
  - 토요타, 혼다와 같은 일본 자동차의 경우 실내 인테리어와 연료 효율, 편안함이 주요 관심사로 보임
- `cluster 1` : 화면, 배터리 수명 등
  - 모바일형이고 엔터테인먼트용 전자제품의 경우 화면 크기와 배터리 수명이 주요 관심사
- `cluster 2` : 방과 서비스 등
  - 호텔 리뷰의 경우 방의 크기나 청소 상태, 직원 서비스, 위치 등이 주요 관심사